# Mechanisms of Action - Kaggle Competition Solution Notebook 

## Loading modules and data 

In [ ]:
import time
import os
import warnings

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer
from sklearn.model_selection import ParameterGrid

from scipy.special import logit
from scipy.stats import kurtosis, skew

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold, MultilabelStratifiedShuffleSplit

In [ ]:
# ignoring multilabel stratified k fold future warning
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
try: # kaggle notebook paths
    train_features = pd.read_csv('../input/lish-moa/train_features.csv')
    train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
    train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
    test_features = pd.read_csv('../input/lish-moa/test_features.csv')

except FileNotFoundError: # local files
    train_features = pd.read_csv('train_features.csv')
    train_targets_scored = pd.read_csv('train_targets_scored.csv')
    train_targets_nonscored = pd.read_csv('train_targets_nonscored.csv')
    test_features = pd.read_csv('test_features.csv')

## Helper Functions

In [ ]:
def preprocessing(X):
    '''
    Combines preprocessing steps into one function
    
    Parameters:
        X (pandas DataFrame): Includes features sig_id, time, dose, and all gene and cell features
        
    Returns:
        X (2D Numpy Array), 
        cp_type (Pandas Series), 
        g_features_ids (1D Numpy Array),
        c_features_ids (1D Numpy Array),
        feature_stats (2D Numpy Array)
    '''
    # replacing time values with smaller values (24,48,72) -> (1,2,3)
    X['cp_time'][X['cp_time']==24] = 0
    X['cp_time'][X['cp_time']==48] = 1
    X['cp_time'][X['cp_time']==72] = 2

    # zero-one encode dose level (high and low)
    X['cp_dose'][X['cp_dose']=='D1'] = 1
    X['cp_dose'][X['cp_dose']=='D2'] = 0

    # remove compound type from training set (ctl_vehicle -> all MOAs == 0)
    cp_type = X['cp_type'].copy()
    X.drop(columns=['cp_type'], inplace=True)

    # remove identifiers from df  
    X.drop(columns=['sig_id'], inplace=True)
    
    # select gene (g) and cell (c) type variables
    g_features_names = [item for item in X.columns if item[0]=='g']
    g_features_ids = np.arange(2,len(g_features_names)+2)

    c_features_names = [item for item in X.columns if item[:2]=='c-']
    c_features_ids = np.arange(g_features_ids[-1],len(c_features_names)+g_features_ids[-1])
    
    X = X.values.astype(float)
    
    # create statistical variables as additional inputs
    feature_stats_list = []
    for ids in [g_features_ids, c_features_ids]:
        feature_stats_list.append(np.mean(X[:,ids], axis=1).reshape(-1,1))
        feature_stats_list.append(np.median(X[:,ids], axis=1).reshape(-1,1))
        feature_stats_list.append(np.std(X[:,ids], axis=1).reshape(-1,1))
        feature_stats_list.append(np.min(X[:,ids], axis=1).reshape(-1,1))
        feature_stats_list.append(np.max(X[:,ids], axis=1).reshape(-1,1))
        feature_stats_list.append(kurtosis(X[:,ids], axis=1).reshape(-1,1))
        feature_stats_list.append(skew(X[:,ids], axis=1).reshape(-1,1))

    feature_stats = np.concatenate(feature_stats_list, axis=1)

    return X, cp_type, g_features_ids, c_features_ids, feature_stats 


In [ ]:
def apply_pca(X, g_features_ids, c_features_ids, scaling=True, g_ratio=0.95, c_ratio=0.95):
    '''
    Applies Principle Component Analysis (PCA)
    
    Parameters:
        X (2D Numpy Array): Containing features,
        g_features_ids (1D Numpy Array): column ids of gene features,
        c_features_ids (1D Numpy Array): column ids of cell features,
        scaling (Boolean): Whether to min-max-scale X before applying PCA,
        g_ratio (Float): Percantage of variance that should be captured with PCA components of genes range (0,1),
        c_ratio (Float): Percantage of variance that should be captured with PCA components of cells range (0,1)
        
    Returns:
        pca_g (sklearn object): object to apply PCA to gene features,
        pca_c (sklearn object): object to apply PCA to cell features        
    '''
    if scaling == True:
        scaler = MinMaxScaler()
        X = scaler.fit_transform(X)
    
    pca_g = PCA(len(g_features_ids))
    pca_g.fit(X[:,g_features_ids])
    components_num_g = np.arange(len(g_features_ids))[np.cumsum(pca_g.explained_variance_ratio_)>=g_ratio][0]

    pca_g = PCA(components_num_g)
    pca_g.fit(X[:,g_features_ids])

    pca_c = PCA(len(c_features_ids))
    pca_c.fit(X[:,c_features_ids])
    components_num_c = np.arange(len(c_features_ids))[np.cumsum(pca_c.explained_variance_ratio_)>=c_ratio][0]

    pca_c = PCA(components_num_c)
    pca_c.fit(X[:,c_features_ids])

    return pca_g, pca_c

## Data Preparation

In [ ]:
### Set root seed
rdm_seed = 17

In [ ]:
# converting chategorical variables and extract cp_type and gene/cell features column ids 
train_features, train_cp_type, g_features_ids, c_features_ids, train_feature_stats = preprocessing(train_features)
test_features, test_cp_type, _, _, test_feature_stats = preprocessing(test_features)

# prepare target labels
train_targets_scored = train_targets_scored.drop(columns=['sig_id']).values.astype(float)


### Principle Component Analysis

Usually it is not recommended to apply PCA on train and test set combined because this allows for information leakage that would not be possible in a later real world application. However, in this Kaggle Competition it is possible to later apply PCA also on the private test set which will be used for the final performance measurement. Thus, it can potentially improve results due to leaking information of the private test set to the training process. Same applies for the quantile transformation further below.

In [ ]:
# combine train and test set for pca variables creation
X_all = np.concatenate((train_features, test_features))
cp_type_all = np.concatenate((train_cp_type, test_cp_type))

X_all = X_all[cp_type_all!='ctl_vehicle']

In [ ]:
# choose gpa ratios 
g_ratio=0.85
c_ratio=0.95

# create pca objects for transforming inputs
pca_module_g, pca_module_c = apply_pca(X_all, g_features_ids, c_features_ids, 
                                       g_ratio=g_ratio, c_ratio=c_ratio)

# create pca variables for genes and cells
pca_g_train_features = pca_module_g.transform(train_features[:,g_features_ids])
pca_g_test_features = pca_module_g.transform(test_features[:,g_features_ids])

pca_c_train_features = pca_module_c.transform(train_features[:,c_features_ids])
pca_c_test_features = pca_module_c.transform(test_features[:,c_features_ids])



In [ ]:
# adding pca variables to features
train_features = np.concatenate((train_features, pca_g_train_features, pca_c_train_features, train_feature_stats), axis=1)
test_features = np.concatenate((test_features, pca_g_test_features, pca_c_test_features, test_feature_stats), axis=1)


### Quantile Transformation (Gaus Transformation)

In [ ]:
# combine train and test set for gaus transformation (works only because applied to private testset later too)
X_all = np.concatenate((train_features, test_features))
cp_type_all = np.concatenate((train_cp_type, test_cp_type))

X_all = X_all[cp_type_all!='ctl_vehicle']

In [ ]:
# transforming gene and cell features as well as their pca counterparts to be normally distributed
gaus_transformer = QuantileTransformer(n_quantiles=100, 
                                        output_distribution='normal', 
                                        random_state=rdm_seed )
gaus_transformer.fit(X_all[:,2:])

train_features[:,2:] = gaus_transformer.transform(train_features[:,2:])
test_features[:,2:] = gaus_transformer.transform(test_features[:,2:])


### Excluding control vehicles from training

The dataset also includes control trials in which no drugs were used, and thus, no Mechanisms of Action (positive labels) can be found. Hence, it makes sense to exclude these rows from training and set all predictions to 0 for the predictions on the testset at the end.  

In [ ]:
X = train_features[train_cp_type!='ctl_vehicle'] 
y = train_targets_scored[train_cp_type!='ctl_vehicle'] 

#### Choosing device to run model on

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### Bias term used to initialize final output layer bias allowing for faster conversion

In [ ]:
output_bias = torch.from_numpy(logit(y.mean(axis=0))).float().to(device)

#### Converting testset to pytorch tensors

In [ ]:
test_features = torch.Tensor(test_features)

## Model Classes and Functions

To make hyperparameter search (including searching for optimal number of layers) easier, I a class for single layers as well as for the combined model. 

In [ ]:
class SingleLayer(nn.Module):
    def __init__(self, input_neurons, output_neurons, dropout_prob, activation, classifier=False):
        '''
        Creates a single layer that is used inside the HyperParamModel class
        
        Parameters:
            input_neurons (int): Number of inputs,
            output_neurons (int): Number of labels, 
            dropout_prob (float): Probability used for dropout, 
            activation (string): Name of activation function that should be used (relu, leaky_relu, elu, parametric_relu),
            classifier (bolean): Whether layer is output (classifier) layer or not
            
        '''
        super().__init__()
        self.activation_dict = nn.ModuleDict([['leaky_relu', nn.LeakyReLU()],
                                              ['relu', nn.ReLU()], 
                                              ['elu', nn.ELU()],
                                              ['parametric_relu', nn.PReLU()]])
        self.classifier = classifier

        if classifier:
            self.linear = nn.Linear(input_neurons, output_neurons)
            self.linear.bias = nn.parameter.Parameter(output_bias.detach().clone())
        else:
            self.activation = self.activation_dict[activation]
            self.dropout = nn.Dropout(dropout_prob)
            
            self.batch_norm = nn.BatchNorm1d(output_neurons)
            self.linear = nn.Linear(input_neurons, output_neurons)
        

    def forward(self, input):
        if self.classifier:
            output = self.linear(input) 
        else:
            output = self.activation(self.dropout(self.batch_norm(self.linear(input))))
        return output

In [ ]:
class HyperParamModel(nn.Module):
    '''
    Main model class
    
    Parameters:
        input_size (int): Number of inputs, 
        hidden_size (int): Number of hidden neurons per layer, 
        output_size (int): Number of output labels, 
        dropout_prob (float): Probability used for dropout,
        num_hidden_layers (int), Number of hidden layers
        activation (string): Name of activation function that should be used (relu, leaky_relu, elu, parametric_relu)
        
    '''
    def __init__(self, input_size, hidden_size, output_size, dropout_prob=0.5, 
                 num_hidden_layers=2, activation='relu'):
        super().__init__()
        self.layers = nn.ModuleList([])
        self.num_layers = num_hidden_layers + 1

        for i in range(self.num_layers):
            # input layer
            if i==0: 
                self.layers.append(SingleLayer(input_size, hidden_size, dropout_prob, activation))
            # output (classifier) layer
            elif i==num_hidden_layers: 
                self.layers.append(SingleLayer(hidden_size, output_size, dropout_prob, activation,
                                   classifier=True))
            # hidden layers
            else: 
                self.layers.append(SingleLayer(hidden_size, hidden_size, dropout_prob, activation))


    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        
        return x


### Loss Function Class

In [ ]:
class Smoothed_BCEWithLogitsLoss(nn.Module):
    def __init__(self, min_clip=1e-6, max_clip=1-1e-6):
        super().__init__()
        self.min_clip = min_clip
        self.max_clip = max_clip

    def forward(self, input, target):
        target = torch.clamp(target, self.min_clip, self.max_clip)
        return F.binary_cross_entropy_with_logits(input, target)

### Train Function

In [ ]:
def train_fct(train_set, batch_size):
    model.train()

    data = DataLoader(train_set, batch_size=batch_size, shuffle=True)    
    
    train_loss = 0

    for inputs, labels in data:
        inputs, labels = inputs.to(device), labels.to(device)        

        optimizer.zero_grad()

        output = model(inputs)

        loss = loss_fct(output, labels)

        loss.backward()

        optimizer.step()
        
        train_loss += loss.item()

    avg_train_loss = train_loss/len(data)

    return avg_train_loss

### Validation Function

In [ ]:
def val_fct(test_set, batch_size):
    model.eval()

    data = DataLoader(test_set, batch_size=batch_size, shuffle=False)    
    
    val_loss = 0

    for inputs, labels in data:
        inputs, labels = inputs.to(device), labels.to(device)        
        with torch.no_grad():
            output = model(inputs)

            loss = val_loss_fct(output, labels)

            val_loss += loss.item()

    avg_val_loss = val_loss/len(data)

    return avg_val_loss

### Function to predict on Testset

In [ ]:
def predict_fct(test_set, batch_size):
    model.eval()

    data = DataLoader(test_set, batch_size=batch_size, shuffle=False)    
    
    predictions = []
    for inputs in data:
        inputs = inputs.to(device)        
        with torch.no_grad():
            # in training sigmoid is included into loss function, thus need to add it here 
            output = torch.sigmoid(model(inputs)) 
            predictions.append(output)

    predictions = torch.cat(predictions).cpu().numpy()
    return predictions

### Function for potential fold specific preprocessing

In [ ]:
def fold_preprocessing(X_train, X_val, y_train, y_val):
    '''
    Preprocessing adjusted to each cross validation fold can be included here.
    Only used for Pytorch tensor transformation
    '''
    X_train = torch.Tensor(X_train)
    y_train = torch.Tensor(y_train)
    X_val = torch.Tensor(X_val)
    y_val = torch.Tensor(y_val)

    return X_train, X_val, y_train, y_val



## Model Training

### Training settings

In [ ]:
# Maximum number of epochs
epochs = 80
# Patience used for early stopping
patience = 4 
# Repetitions of CV scheme
repetitions = 3
# Number of folds to devide training set into
num_folds = 10
# Select whether to print information on fewer epochs during training
num_epochs_to_print = 2
# Whether to shuffle folds in multi label stratified K fold CV 
shuffle_folds = True

### Model parameters

In [ ]:
# hyper parameters to search over
hyper_params = {'num_hidden_layers': [2],
                'batch_size': [128],
                'learning_rate': [0.005],
                'hidden_size': [256],
                'dropout_prob': [0.5],
                'activation': ['leaky_relu'],
                'label_smoothing':[5e-5],
                'weight_decay': [1e-6, 5e-7, 3e-7],
                'lr_plateau_factor': [0.1]} 

# saving best parameters
default_params = {'num_hidden_layers': [2],
                  'batch_size': [128],
                  'learning_rate': [0.005],
                  'hidden_size': [256],
                  'dropout_prob': [0.5],
                  'activation': ['leaky_relu'],
                  'label_smoothing':[5e-5],
                  'weight_decay': [1e-6],
                  'lr_plateau_factor': [0.1]}

# select parameter dictionary (hyper_params/default_params)
param_grid = ParameterGrid(default_params)
# select number of searches | len(param_grid): exhaustive search; int < len(param_grid): random search 
grid_search_iterations = len(param_grid) 

random_grid_ids = np.random.choice(len(param_grid), size=grid_search_iterations, replace=False)



#### Loss function for validation (no label smoothing)

In [ ]:
val_loss_fct = nn.BCEWithLogitsLoss().to(device)

### Running Cross Validation

In [ ]:
# saving best model checkpoints
model_checkpoints_all = []
model_checkpoints_epochs = []
# saving results
hyper_search_losses = np.zeros(grid_search_iterations)
hyper_search_params = []
hyper_search_best_epochs = []
hyper_search_results = []

# looping over selections of hyper parameters
for search_i, grid_id in enumerate(random_grid_ids):
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')    
    print(f'\t{search_i+1}/{grid_search_iterations} hyper parameter combinations to be tested')
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    
    random_params = param_grid[grid_id]
    hyper_search_params.append(random_params)
    
    num_hidden_layers = random_params['num_hidden_layers']
    batch_size = random_params['batch_size']
    learning_rate = random_params['learning_rate']
    hidden_size = random_params['hidden_size']
    dropout_prob = random_params['dropout_prob']
    activation = random_params['activation']
    label_smoothing_cuts = random_params['label_smoothing']
    weight_decay = random_params['weight_decay']
    lr_plateau_factor = random_params['lr_plateau_factor']

    loss_fct = Smoothed_BCEWithLogitsLoss(min_clip=label_smoothing_cuts, max_clip=1-label_smoothing_cuts).to(device)
    
    print('Hyper Parameters:')
    print(f'Weight decay: {weight_decay}') 
    
    # losses of all repetitions
    all_train_losses = np.array([]).reshape(0,epochs)
    all_val_losses = np.array([]).reshape(0,epochs)
    
    all_best_epochs = []

    train_times = []
    
    test_predictions_all = []
    
    # looping over random seeds
    for rep in range(repetitions):
        print('\t----------------------------------------------------')
        print(f'\t------------------- Repitition {rep+1} -------------------')
        print('\t----------------------------------------------------')
        
        # different seed per iteration
        rep_seed = rdm_seed+rep
        torch.manual_seed(rep_seed)
        
        strat_cv = MultilabelStratifiedKFold(n_splits=num_folds, 
                                            shuffle=shuffle_folds,
                                            random_state=rep_seed)
        
        # losses per fold
        train_losses = np.ones((num_folds,epochs))
        val_losses = np.ones((num_folds,epochs))
        

        start_time = time.time()
        # iterating over folds
        for i, (train_ids, val_ids) in enumerate(strat_cv.split(X, y)):
            print(f'++++++++++++++++++++++++++ Fold Number {i+1:2} ++++++++++++++++++++++++++')
            # CV split 
            X_train = X[train_ids]
            y_train = y[train_ids]
            X_val = X[val_ids]
            y_val = y[val_ids]

            # converting arrays into tensors
            X_train, X_val, y_train, y_val = fold_preprocessing(X_train, X_val, y_train, y_val)

            train_set = TensorDataset(X_train, y_train)
            val_set = TensorDataset(X_val, y_val)

            # initilize model
            input_size = X_train.shape[1]
            output_size = y_train.shape[1]

            model = HyperParamModel(input_size, hidden_size, output_size, 
                                    dropout_prob=dropout_prob, num_hidden_layers=num_hidden_layers,
                                    activation=activation).to(device)

            optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay=weight_decay)
            
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=lr_plateau_factor, patience=1, 
                                                                   threshold=1e-6, verbose=True)
            # alternitive scheduler
            #cosine_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=scheduler_steps, 
            #                                                              eta_min=min_learning_rate, last_epoch=-1)
            

            # list with model checkpoints for all epochs per fold
            model_checkpoints_fold = []
            # lists with losses per epoch
            train_loss_list = []
            val_loss_list = []

            patience_counter = 0
            
            # Model training
            for epoch in range(epochs):                
                #print(f'Scheduled LR: {cosine_scheduler.get_lr()[0]:.4}')
                train_loss = train_fct(train_set, batch_size)
                val_loss = val_fct(val_set, batch_size)
                
                if num_epochs_to_print:
                    if epoch < num_epochs_to_print:
                        print(f'Epoch {epoch+1:2} | Train Loss: {train_loss:.6}  \t  Validation Loss: {val_loss:.6}')
                else:
                    print(f'Epoch {epoch+1:2} | Train Loss: {train_loss:.6}  \t  Validation Loss: {val_loss:.6}')
                
                # for early stopping
                if epoch == 0:
                    pass
                elif val_loss > min(val_loss_list):
                    patience_counter += 1 
                else:
                    patience_counter = 0

                train_loss_list.append(train_loss)
                val_loss_list.append(val_loss)
                
                # apply early stopping
                if patience_counter == patience:
                    if num_epochs_to_print:
                        print(f'Epoch {epoch+1:2} | Train Loss: {train_loss:.6}  \t  Validation Loss: {val_loss:.6}')
                    print(f'---- Early stopping with best val-loss {np.min(val_loss_list):.6} in epoch {np.argmin(val_loss_list)+1} ----')
                    break

                scheduler.step(val_loss)
                #cosine_scheduler.step()    
                #print(f'Scheduled LR: {cosine_scheduler.get_lr()[0]:.4}')

                ### save model parameters as checkpoint after each epoch
                model_checkpoints_fold.append(model.state_dict().copy())


            ### save only best model checkpoint after certain epoch
            best_epoch_fold = np.argmin(val_loss_list)
            all_best_epochs.append(best_epoch_fold)
            model_checkpoints_all.append(model_checkpoints_fold[best_epoch_fold])
            model_checkpoints_epochs.append(best_epoch_fold)            
            
            # save losses of current fold
            train_losses[i,:len(train_loss_list)] = np.array(train_loss_list)
            val_losses[i,:len(train_loss_list)] = np.array(val_loss_list)

        all_train_losses = np.concatenate((all_train_losses, train_losses))
        all_val_losses = np.concatenate((all_val_losses, val_losses))
        
        end_time = time.time()
        duration = int(end_time-start_time)
        print('-------------------------')
        print(f'Training took {duration} seconds')
        print('-------------------------')
        train_times.append(duration)
    # calculate mean of loss of each fold (out of fold loss)
    oof_loss = np.min(all_val_losses, axis=1).mean().round(8)
    # calculate std of repetitions
    overall_std = np.min(all_val_losses, axis=1).std().round(8)
    over_repetition_means_std = np.min(all_val_losses, axis=1).reshape(-1, repetitions).mean(axis=0).std().round(8)
    # add oof loss to array
    hyper_search_losses[search_i] = oof_loss
    # save best performing epoch of each fold
    hyper_search_best_epochs.append(all_best_epochs)
    # add oof loss to parameters for results dataframe    
    random_params['oof_loss'] = oof_loss
#    random_params['snap_oof_loss'] = snap_oof_loss
    random_params['overall_std'] = overall_std
    random_params['over_repetition_means_std'] = over_repetition_means_std

    random_params['time per seed'] = np.mean(train_times).round(0)
    hyper_search_results.append(random_params)
    
    test_predictions_all = np.array(test_predictions_all)


In [ ]:
hyper_search_results = pd.DataFrame(hyper_search_results)

hyper_search_results.sort_values('oof_loss', inplace=True)

hyper_search_results.to_csv('hyper_search_results.csv', index=False)

hyper_search_results

## Test Submission

#### Load submission file

In [ ]:
try:
    sample_submission = pd.read_csv('../input/lish-moa/sample_submission.csv')
except FileNotFoundError: 
    sample_submission = pd.read_csv('sample_submission.csv')


#### Predict with each model saved during CV

In [ ]:
test_predictions_checkpoints = np.zeros((len(model_checkpoints_all), test_features.shape[0], y.shape[1]))

for i, checkpoint in enumerate(model_checkpoints_all):
    model.load_state_dict(checkpoint)
    test_predictions = predict_fct(test_features, batch_size)

    test_predictions_checkpoints[i,:,:] = test_predictions

In [ ]:
test_predictions = test_predictions_checkpoints.mean(axis=0)

#### Clipping pridictions because of high penalty for highly confident wrong predictions due to log loss metric

In [ ]:
test_predictions = np.clip(test_predictions, 0.001, 0.999)

#### Change all predictions to 0 for control vehicles 

In [ ]:
test_predictions[test_cp_type=='ctl_vehicle'] = np.zeros(test_predictions.shape[1])

#### Fill and save submission file

In [ ]:
sample_submission.iloc[:,1:] = test_predictions
submission = sample_submission

In [ ]:
submission.to_csv('submission.csv', index=False)